<a href="https://colab.research.google.com/github/GQ131/24-Frames-in-Data-Film-Data-Set-Analysis/blob/main/Federated_Referral_PoC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# ================== Federated Referral PoC (Option A) ==================
# Rural-only, ZIP→Lat/Long/County join, loop preview, Tableau-ready CSV
# Time window: Jan 1–Dec 31, 2024 | Program: RSEP
# ======================================================================
from google.colab import files
import pandas as pd, numpy as np, uuid, re
from datetime import timedelta


uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving ALRB_cases.xlsx to ALRB_cases (7).xlsx
Saving California_Zip_Lat_Lng_City_State_County.csv to California_Zip_Lat_Lng_City_State_County (8).csv
Saving DOSH_cases.xlsx to DOSH_cases (7).xlsx
Saving LCO_cases.xlsx to LCO_cases (7).xlsx
User uploaded file "ALRB_cases (7).xlsx" with length 32512 bytes
User uploaded file "California_Zip_Lat_Lng_City_State_County (8).csv" with length 129174 bytes
User uploaded file "DOSH_cases (7).xlsx" with length 26364811 bytes
User uploaded file "LCO_cases (7).xlsx" with length 680909 bytes


In [9]:
# ---- Update these paths to your uploaded files in Colab ----
ZIP_LATLONG_PATH = "/content/California_Zip_Lat_Lng_City_State_County.csv"  # has: Country, State/Province, ZIP Code/Postcode, Latitude, Longitude, County, City
DLSE_PATH        = "/content/LCO_cases.xlsx"
ALRB_PATH        = "/content/ALRB_cases.xlsx"
CALOSHA_PATH     = "/content/DOSH_cases.xlsx"


In [10]:
import os
os.listdir('/content')

['.config',
 'DOSH_cases (3).xlsx',
 'California_Zip_Lat_Lng_City_State_County (2).csv',
 'LCO_cases (2).xlsx',
 'ALRB_cases (2).xlsx',
 'California_Zip_Lat_Lng_City_State_County (8).csv',
 'California_Zip_Lat_Lng_City_State_County (6).csv',
 'California_Zip_Lat_Lng_City_State_County (5).csv',
 'California_Zip_Lat_Lng_City_State_County (1).csv',
 'LCO_cases (7).xlsx',
 'ALRB_cases.xlsx',
 'DOSH_cases (6).xlsx',
 'ALRB_cases (4).xlsx',
 'ALRB_cases (6).xlsx',
 'DOSH_cases (5).xlsx',
 'DOSH_cases (2).xlsx',
 'California_Zip_Lat_Lng_City_State_County (4).csv',
 'DOSH_cases (7).xlsx',
 'California_Zip_Lat_Lng_City_State_County (7).csv',
 'ALRB_cases (7).xlsx',
 'ALRB_cases (5).xlsx',
 'California_Zip_Lat_Lng_City_State_County (3).csv',
 'DOSH_cases (1).xlsx',
 'LCO_cases (6).xlsx',
 'LCO_cases (5).xlsx',
 'DOSH_cases (4).xlsx',
 'tableau_referrals_poc_rural_only.csv',
 'LCO_cases (3).xlsx',
 'LCO_cases (1).xlsx',
 'DOSH_cases.xlsx',
 'LCO_cases (4).xlsx',
 'ALRB_cases (1).xlsx',
 'LCO_case

In [11]:
# ---- PoC knobs ----
ALL_AGENCIES = ["DIR","Cal/OSHA","DLSE","ALRB","EDD"]
PROGRAM_VALUE = "RSEP"
LOOP_RATE = 0.08                       # ~8% second-hop preview
MIN_SEND_LAG, MAX_SEND_LAG = 0, 7      # days intake -> referral
MIN_STATUS_LAG, MAX_STATUS_LAG = 3, 21 # days referral -> first update
TARGET_MAX_ROWS = 600
YEARS = {2023, 2024}

In [12]:
# Canonical output columns (used everywhere to keep shape stable)
COLS_CANON = [
    "Referral_UID","Complaint_Intake_Date","Referral_Sent_Date",
    "Origin_Agency","Destination_Agency","Program",
    "County","ZIP","Latitude","Longitude","Is_Rural_Worker",
    "Referral_Status","Response_Completed","Days_To_First_Status_Update",
    "Loop_Destination_Agency","Loop_Referral_Sent_Date",
]

In [13]:
# Curated rural counties list (tune if needed)
RURAL_COUNTIES = {
    "Amador","Calaveras","Colusa","Del Norte","El Dorado","Glenn","Humboldt","Imperial","Inyo",
    "Kings","Lake","Lassen","Madera","Mariposa","Mendocino","Merced","Modoc","Mono","Napa",
    "Nevada","Placer","Plumas","San Benito","Shasta","Sierra","Siskiyou","Sutter","Tehama",
    "Trinity","Tulare","Tuolumne","Yuba","Kern", "Fresno", "Stanislaus"
}

# ---- Helpers ----
def zip5(z):
    if pd.isna(z):
        return ""
    s = str(z).strip()
    m = re.search(r"(\d{5})", s)
    return m.group(1) if m else ""

def coerce_date(s):
    return pd.to_datetime(s, errors="coerce")

def normalize_status_from_closed(closed_date, status_hint=None):
    if pd.notna(closed_date):
        return "Closed"
    if status_hint is not None and isinstance(status_hint, str):
        sl = status_hint.lower()
        if any(k in sl for k in ["close","settle","resolved","dismiss"]):
            return "Closed"
        if any(k in sl for k in ["progress","investigat","acknow"]):
            return "In Progress"
    return "In Progress"

def choose_destination(origin, weights):
    w = dict(weights) if weights else {origin: 0.6}
    for a in ALL_AGENCIES:
        w.setdefault(a, 0.0)
    agencies, probs = zip(*w.items())
    probs = np.array(probs, dtype=float)
    probs = probs / probs.sum() if probs.sum() else np.ones_like(probs)/len(probs)
    return np.random.choice(agencies, p=probs)

def pick_status_and_completion(closed_flag):
    if closed_flag:
        return "Closed","Y"
    return np.random.choice(["In Progress","Open"], p=[0.75,0.25]), np.random.choice(["Y","N"], p=[0.5,0.5])

def load_xlsx(path, sheet=None, header=None):
    if sheet is None:
        # read all sheets with default header=0
        x = pd.read_excel(path, sheet_name=None)
        frames = []
        for name, df in x.items():
            df = df.copy()
            df["__sheet__"] = name
            frames.append(df)
        return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()
    # default to header=0 unless a custom header row is provided
    return pd.read_excel(path, sheet_name=sheet, header=0 if header is None else header)



In [14]:
# ---- Load ZIP→Lat/Long/County lookup and normalize ZIP ----
geo = pd.read_csv(ZIP_LATLONG_PATH, dtype=str, keep_default_na=False)
zip_col = "ZIP Code/Postcode"; lat_col = "Latitude"; lon_col = "Longitude"; county_col = "County"
geo = geo.rename(columns={zip_col:"ZIP5", lat_col:"Latitude", lon_col:"Longitude", county_col:"GeoCounty"})
geo["ZIP5"] = geo["ZIP5"].apply(zip5)
geo = geo[(geo["ZIP5"]!="") & (geo["GeoCounty"]!="")]
geo = geo[["ZIP5","Latitude","Longitude","GeoCounty"]].drop_duplicates()

In [15]:
# ---- Agency configs (based on your columns) ----
CONFIG = {
    "DLSE": {
        "path": DLSE_PATH, "origin": "DLSE", "sheet": 0,
        "cols": {
            "intake_date": "Case Management: Created Date",
            "closed_date": "Case Closed Date",
            "county": None,
            "zip": "Employer: Primary Zip/Postal Code",
            "status_hint": None,
            "case_id": "Case Management: DIR Case Name",
        },
        "routing_weights": {"DLSE": 0.60, "Cal/OSHA": 0.20, "DIR": 0.10, "ALRB": 0.05, "EDD": 0.05},
    },
    "ALRB": {
        "path": ALRB_PATH, "origin": "ALRB", "sheet": None,
        "cols": {
            "intake_date": "Filed Date",
            "closed_date": None,
            "county": "County",
            "zip": None,
            "status_hint": "Outcome",
            "case_id": "Case Number",
        },
        "routing_weights": {"ALRB": 0.65, "DLSE": 0.10, "Cal/OSHA": 0.10, "DIR": 0.10, "EDD": 0.05},
    },

    # In your CONFIG block for Cal/OSHA:
    "Cal/OSHA": {
        "path": CALOSHA_PATH, "origin": "Cal/OSHA",
        "sheet": "Complaints",
        "header_row": 14,
        "cols": {
            "intake_date": "Open Conf Date",
            "closed_date": "Closed Date",
            "county": "GeoCounty",   # <- use GeoCounty
            "zip": "GeoZip",         # <- use GeoZip
            "status_hint": None,
            "case_id": "RID",
        },
        "routing_weights": {"Cal/OSHA": 0.50, "DLSE": 0.30, "DIR": 0.10, "ALRB": 0.05, "EDD": 0.05},
    }
}

In [16]:
# ---- Builder per agency (handles blanks, uneven sizes, 2024 filter) ----

def empty_out():
    """Return an empty dataframe with canonical columns."""
    return pd.DataFrame({c: pd.Series(dtype="object") for c in COLS_CANON})[COLS_CANON]

def build_from_agency(cfg):
    df_raw = load_xlsx(cfg["path"], cfg.get("sheet"), cfg.get("header_row"))
    if df_raw is None or df_raw.empty:
        return empty_out()

    c = cfg["cols"]; ori = cfg["origin"]

    # graceful fallbacks
    if c.get("county") and c["county"] not in df_raw.columns and "County" in df_raw.columns:
        c["county"] = "County"
    if c.get("zip") and c["zip"] not in df_raw.columns:
        if "Site  Zip" in df_raw.columns:
            c["zip"] = "Site  Zip"

    # Required intake date column must exist
    if not c.get("intake_date") or c["intake_date"] not in df_raw.columns:
        return empty_out()

    intake = coerce_date(df_raw[c["intake_date"]])
    closed = coerce_date(df_raw[c["closed_date"]]) if c.get("closed_date") and c["closed_date"] in df_raw.columns else pd.Series(pd.NaT, index=df_raw.index)
    status_hint = df_raw[c["status_hint"]] if c.get("status_hint") and c["status_hint"] in df_raw.columns else None

    # Years
    mask_years = intake.notna() & (intake.dt.year.isin(YEARS))
    if not mask_years.any():
        return empty_out()

    df = df_raw.loc[mask_years].copy()
    intake = intake.loc[mask_years]
    closed = closed.loc[mask_years] if isinstance(closed, pd.Series) else pd.Series(pd.NaT, index=df.index)
    if isinstance(status_hint, pd.Series):
        status_hint = status_hint.loc[mask_years]

    out = pd.DataFrame(index=df.index)
    out["Referral_UID"] = [str(uuid.uuid4()) for _ in range(len(df))]
    out["Complaint_Intake_Date"] = intake.dt.date

    # Referral sent date
    send_lag = np.random.randint(MIN_SEND_LAG, MAX_SEND_LAG+1, size=len(df))
    ref_sent = (intake + pd.to_timedelta(send_lag, unit="D"))
    out["Referral_Sent_Date"] = ref_sent.dt.date
    out["Origin_Agency"] = ori
    out["Program"] = PROGRAM_VALUE

    # ZIP normalized → 5-digit
    if c.get("zip") and c["zip"] in df.columns:
        out["ZIP"] = df[c["zip"]].apply(zip5)
    else:
        out["ZIP"] = ""

    # County: direct if present, else infer from ZIP via geo
    if c.get("county") and c["county"] in df.columns:
        county_series = df[c["county"]].fillna("").astype(str).str.strip()
    else:
        county_series = pd.Series([""]*len(df), index=df.index)

    # Merge ZIP→(County, Lat, Lon)
    z = out[["ZIP"]].copy().rename(columns={"ZIP":"ZIP5"})
    merged = z.merge(geo, how="left", on="ZIP5")

    # *** Align merged to out's index to avoid shape mismatches ***
    merged.index = out.index

    # Lat/Lon from ZIP master (will be blank if ZIP missing/not found)
    out["Latitude"]  = merged["Latitude"].fillna("")
    out["Longitude"] = merged["Longitude"].fillna("")

    # ----- County resolution (Policy: B and 1) -----
    # Prefer County from ZIP master when ZIP is present; otherwise keep agency County.
    county_from_agency = (
        county_series.reindex(out.index)   # <-- align to out
                    .fillna("")
                    .astype(str).str.strip()
    )
    county_from_zip = (
        merged["GeoCounty"].fillna("")
                          .astype(str).str.strip()
    )

    out["County"] = np.where(
        (out["ZIP"] != "") & (county_from_zip != ""),
        county_from_zip,
        county_from_agency
    )

    # Rural rule
    out["Is_Rural_Worker"] = out["County"].apply(lambda cn: "Y" if cn in RURAL_COUNTIES else "N")

    # Jurisdiction filter: rural only
    out = out[out["Is_Rural_Worker"]=="Y"].copy()
    if out.empty:
        return empty_out()

    # Destination selection
    weights = cfg.get("routing_weights", {})
    out["Destination_Agency"] = [choose_destination(ori, weights) for _ in range(len(out))]

    # Status & completion
    closed_map = pd.Series(closed.values, index=df.index).loc[out.index]
    if isinstance(status_hint, pd.Series):
        status_hint_map = pd.Series(status_hint.values, index=df.index).loc[out.index]
    else:
        status_hint_map = pd.Series([None]*len(out), index=out.index)

    norm_status = [
        normalize_status_from_closed(closed_map.loc[i], status_hint_map.loc[i])
        for i in out.index
    ]
    is_closed = [s == "Closed" for s in norm_status]
    final_status, completed = [], []
    for cl in is_closed:
        s, comp = pick_status_and_completion(cl)
        if cl: s, comp = "Closed","Y"
        final_status.append(s); completed.append(comp)
    out["Referral_Status"] = final_status
    out["Response_Completed"] = completed

    # Days to first update
    out["Days_To_First_Status_Update"] = np.random.randint(MIN_STATUS_LAG, MAX_STATUS_LAG+1, size=len(out))

    # Loop preview (~8%)
    n = len(out)
    mask_loop = np.random.rand(n) < LOOP_RATE
    loop_dest, loop_date = [], []
    for idx, m in zip(out.index, mask_loop):
        if not m:
            loop_dest.append(""); loop_date.append("")
            continue
        first_dest = out.loc[idx, "Destination_Agency"]
        choices = [a for a in ALL_AGENCIES if a != first_dest]
        loop_dest.append(np.random.choice(choices))
        d0 = pd.to_datetime(out.loc[idx, "Referral_Sent_Date"])
        loop_date.append((d0 + timedelta(days=int(np.random.randint(2,11)))).date())
    out["Loop_Destination_Agency"] = loop_dest
    out["Loop_Referral_Sent_Date"] = loop_date

    # Ensure canonical order
    for ccc in COLS_CANON:
        if ccc not in out.columns:
            out[ccc] = ""
    return out[COLS_CANON]



In [17]:
# ------------------ Build all and export ------------------
frames = []
for key, cfg in CONFIG.items():
    print(f"Processing: {key}")
    frames.append(build_from_agency(cfg))

# If everything came back empty, still produce a CSV with headers
if len(frames) == 0:
    final = empty_out()
else:
    final = pd.concat(frames, ignore_index=True) if any(not f.empty for f in frames) else empty_out()

# Cap size for PoC if very large; sort by date for nicer Tableau behavior
if len(final) > TARGET_MAX_ROWS:
    final = final.sample(TARGET_MAX_ROWS, random_state=42).sort_values("Referral_Sent_Date")

# --- FINAL TYPE HYGIENE BEFORE EXPORT ---

# ZIP as text
final["ZIP"] = final["ZIP"].astype(str)

# Lat/Lon as float when possible, else blank string
for col in ["Latitude","Longitude"]:
    final[col] = pd.to_numeric(final[col], errors="coerce")
    final[col] = final[col].apply(lambda x: "" if pd.isna(x) else x)

# Dates to ISO strings
for col in ["Complaint_Intake_Date","Referral_Sent_Date","Loop_Referral_Sent_Date"]:
    final[col] = pd.to_datetime(final[col], errors="coerce").dt.strftime("%Y-%m-%d")

OUT_PATH = "/content/tableau_referrals_poc_rural_only.csv"
final.to_csv(OUT_PATH, index=False)
print(f"\n✅ Wrote {OUT_PATH} with {len(final)} rural rows.\n")



Processing: DLSE
Processing: ALRB
Processing: Cal/OSHA

✅ Wrote /content/tableau_referrals_poc_rural_only.csv with 600 rural rows.



In [18]:
# ====================== (A) Preview Summary ==========================
# Month & Origin counts, completion rate, avg days, loops count
def month(d):
    try:
        return pd.to_datetime(d).strftime("%Y-%m")
    except Exception:
        return ""

summary = final.copy()
summary["Month"] = summary["Referral_Sent_Date"].apply(month)
summary["Loop_Flag"] = summary["Loop_Destination_Agency"].apply(lambda x: 1 if isinstance(x, str) and x.strip() != "" else 0)

grouped = summary.groupby(["Month","Origin_Agency"], dropna=False).agg(
    referrals=("Referral_UID","count"),
    completed_pct=("Response_Completed", lambda s: (s.eq("Y").mean()*100).round(1) if len(s) else 0),
    avg_days_to_update=("Days_To_First_Status_Update","mean"),
    loops=("Loop_Flag","sum")
).reset_index()

print("=== Preview Summary (by Month & Origin_Agency) ===")
print(grouped.sort_values(["Month","Origin_Agency"]).to_string(index=False))




=== Preview Summary (by Month & Origin_Agency) ===
  Month Origin_Agency  referrals  completed_pct avg_days_to_update  loops
2023-01          DLSE         12          100.0              13.25      1
2023-02          DLSE         13           92.3          11.769231      3
2023-03          ALRB          2            0.0                8.0      0
2023-03          DLSE         22           81.8          11.954545      0
2023-04          DLSE         14           57.1          11.071429      0
2023-05          ALRB          2           50.0               10.5      0
2023-05          DLSE         14           85.7          14.285714      2
2023-06          ALRB          3            0.0          16.666667      2
2023-06          DLSE         14           85.7          12.357143      1
2023-07          ALRB          3           33.3                9.0      1
2023-07          DLSE         28           89.3          12.571429      0
2023-08          ALRB          1            0.0              

In [19]:
# Overall quick KPIs
overall = {
    "Total referrals": len(final),
    "Completion rate (%)": round(final["Response_Completed"].eq("Y").mean()*100, 1) if len(final) else 0,
    "Avg days to first update": round(final["Days_To_First_Status_Update"].mean(), 1) if len(final) else 0,
    "Loop share (%)": round((summary["Loop_Flag"].mean()*100), 1) if len(final) else 0,
}
print("\n=== Overall KPIs ===")
for k,v in overall.items():
    print(f"{k}: {v}")

# ====================== (C) Sample Validation ========================
print("\n=== Sample 10 rows ===")
print(final.head(10).to_string(index=False))


=== Overall KPIs ===
Total referrals: 600
Completion rate (%): 73.8
Avg days to first update: 12.2
Loop share (%): 8.5

=== Sample 10 rows ===
                        Referral_UID Complaint_Intake_Date Referral_Sent_Date Origin_Agency Destination_Agency Program   County   ZIP  Latitude  Longitude Is_Rural_Worker Referral_Status Response_Completed Days_To_First_Status_Update Loop_Destination_Agency Loop_Referral_Sent_Date
a089764b-ee58-4e85-a266-cf25b9131722            2023-01-06         2023-01-12          DLSE               DLSE    RSEP     Lake 95467  38.80453 -122.54948               Y     In Progress                  Y                          15                                             NaN
b7219e84-c0b5-4eab-a68d-930702577f81            2023-01-09         2023-01-13          DLSE               DLSE    RSEP    Kings 93245  36.28614 -119.83405               Y          Closed                  Y                          17                                             NaN
643bc44e-2

In [27]:
OUT_PATH = "/content/tableau_referrals_poc_rural_only.csv"
final.to_csv(OUT_PATH, index=False)
print(f"\n✅ Wrote {OUT_PATH} with {len(final)} rural rows.\n")

from google.colab import files
files.download(OUT_PATH)



✅ Wrote /content/tableau_referrals_poc_rural_only.csv with 600 rural rows.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import pandas as pd

# List sheets
xls = pd.ExcelFile(CALOSHA_PATH)
print("Sheets:", xls.sheet_names)

# Peek columns from the sheet you're using (adjust index if needed)
df_test = pd.read_excel(CALOSHA_PATH, sheet_name=0)
print("Sheet[0] columns:", list(df_test.columns))


Sheets: ['Notes', 'References - Lists', 'Sheet1', 'Sheet3', 'Complaints', 'Industries- Tables', 'Industries - Charts', 'All Regions - Charts', 'Complaint vs Insp Coded', 'Heat Referrals', 'Comparison by NAICS', 'Comparison by YR', 'Other Inspections Coded', 'Totals by Ag Subsector', 'Sheet2', 'Region 1 - Charts', 'Region 2 - Charts', 'Region 3 - Charts', 'Region 4 - Charts', 'All Regions - Charts (2)']
Sheet[0] columns: ['UPA One Liner Detail']


In [22]:
import pandas as pd

xls = pd.ExcelFile(CALOSHA_PATH)

def find_col(df, targets):
    norm = {c: c.strip().lower().replace(".", "").replace("_", "").replace("  ", " ")
            for c in df.columns}
    for t in targets:
        tnorm = t.strip().lower().replace(".", "").replace("_", "").replace("  ", " ")
        for c, cnorm in norm.items():
            if cnorm == tnorm:
                return c
    return None

intake_targets = [
    "Open Conf Date","Open Conf. Date","Open Conference Date",
    "Open ConfDate","Open Conference Dt","OpenConfDate"
]
zip_targets = ["GeoZip","Geo Zip","Site Zip","Site  Zip"]
county_targets = ["GeoCounty","Geo County","County"]

candidates = []
for s in xls.sheet_names:
    try:
        df = pd.read_excel(CALOSHA_PATH, sheet_name=s)
        ic = find_col(df, intake_targets)
        zc = find_col(df, zip_targets)
        cc = find_col(df, county_targets)
        score = (ic is not None) + (zc is not None) + (cc is not None)
        if score >= 2:  # needs intake + (zip or county)
            candidates.append((s, ic, zc, cc, score, len(df.columns)))
    except Exception as e:
        pass

# sort: best score first, then more columns (more likely the raw table)
candidates.sort(key=lambda x: (-x[4], -x[5]))
print("Candidates (best first):")
for s, ic, zc, cc, score, ncols in candidates[:8]:
    print(f"  • Sheet='{s}' | intake='{ic}' | zip='{zc}' | county='{cc}' | score={score} | ncols={ncols}")

if not candidates:
    print("No suitable sheet found. Try 'Complaints', 'Sheet1', or 'Sheet3' manually.")


Candidates (best first):
No suitable sheet found. Try 'Complaints', 'Sheet1', or 'Sheet3' manually.


In [23]:
import pandas as pd

LIKELY_SHEETS = ["Complaints", "Sheet1", "Sheet3"]

def find_col(df, targets):
    norm = {c: c.strip().lower().replace(".", "").replace("_", "").replace("  "," ")
            for c in df.columns}
    for t in targets:
        tnorm = t.strip().lower().replace(".", "").replace("_", "").replace("  "," ")
        for c, cnorm in norm.items():
            if cnorm == tnorm:
                return c
    return None

intake_candidates = ["Open Conf Date","Open Conf. Date","Open Conference Date","Open ConfDate","Open Conference Dt","OpenConfDate"]
zip_candidates    = ["GeoZip","Geo Zip","Site Zip","Site  Zip"]
county_candidates = ["GeoCounty","Geo County","County"]

results = []
for s in LIKELY_SHEETS:
    try:
        df = pd.read_excel(CALOSHA_PATH, sheet_name=s)
        ic = find_col(df, intake_candidates)
        zc = find_col(df, zip_candidates)
        cc = find_col(df, county_candidates)
        score = (ic is not None) + (zc is not None) + (cc is not None)
        results.append((s, ic, zc, cc, score, len(df), len(df.columns)))
    except Exception as e:
        results.append((s, None, None, None, -1, 0, 0))

print("Sheet scan (best first):")
for s, ic, zc, cc, score, nrows, ncols in sorted(results, key=lambda x: (-x[4], -x[5], -x[6])):
    print(f"  • {s:12s} | intake={ic} | zip={zc} | county={cc} | score={score} | rows={nrows} | cols={ncols}")


Sheet scan (best first):
  • Complaints   | intake=None | zip=None | county=None | score=0 | rows=3059 | cols=116
  • Sheet3       | intake=None | zip=None | county=None | score=0 | rows=57 | cols=7
  • Sheet1       | intake=None | zip=None | county=None | score=0 | rows=55 | cols=1


In [24]:
import pandas as pd, re

SHEET = "Complaints"  # likely the raw table

# patterns we expect to see somewhere in the header row
PATTERNS = {
    "intake":  re.compile(r"open\s*conf.*date", re.I),
    "zip":     re.compile(r"\bgeo\s*zip\b|\bsite\s*zip\b", re.I),
    "county":  re.compile(r"\bgeo\s*county\b|\bcounty\b", re.I),
}

# 1) read a small slice first without headers to find which row contains the header
peek = pd.read_excel(CALOSHA_PATH, sheet_name=SHEET, header=None, nrows=30)
header_row = None
for i in range(len(peek)):
    row_vals = [str(x) for x in peek.iloc[i].tolist()]
    joined = " | ".join(row_vals)
    if any(p.search(joined) for p in PATTERNS.values()):
        header_row = i
        break

print("Detected header row:", header_row)

# 2) re-read using that header row
if header_row is None:
    raise ValueError("Could not auto-detect a header row. Try increasing nrows or inspecting manually.")

df_calo = pd.read_excel(CALOSHA_PATH, sheet_name=SHEET, header=header_row)

# 3) normalize headers (for matching & sanity)
def norm(s):
    s = str(s)
    s = s.replace("\xa0"," ")  # non-breaking spaces
    s = re.sub(r"\s+", " ", s)
    return s.strip()

df_calo.columns = [norm(c) for c in df_calo.columns]

# 4) find the actual column names by regex matching on the cleaned headers
def find_col_by_regex(cols, patt):
    for c in cols:
        if patt.search(c):
            return c
    return None

intake_col = find_col_by_regex(df_calo.columns, PATTERNS["intake"])
zip_col    = find_col_by_regex(df_calo.columns, PATTERNS["zip"])
county_col = find_col_by_regex(df_calo.columns, PATTERNS["county"])

print("Resolved columns →")
print("  intake:", intake_col)
print("  zip   :", zip_col)
print("  county:", county_col)

# (optional) quick year counts to confirm we’ll retain rows
years = pd.to_datetime(df_calo[intake_col], errors="coerce").dt.year
print("Counts by year:\n", years.value_counts().sort_index())


Detected header row: 14
Resolved columns →
  intake: Open Conf Date
  zip   : Site Zip
  county: Site County
Counts by year:
 Open Conf Date
2015.0    273
2016.0    266
2017.0    315
2018.0    293
2019.0    315
2020.0    133
2021.0    246
2022.0    330
2023.0    144
Name: count, dtype: int64


In [25]:
import re

# Read with header row 15 (0-indexed 14)
raw = pd.read_excel(CALOSHA_PATH, sheet_name="Complaints", header=14)

# Normalize headers (collapse weird spaces/punctuation)
raw.columns = (
    raw.columns.astype(str)
       .str.replace("\xa0"," ", regex=False)
       .str.replace(r"\s+"," ", regex=True)
       .str.strip()
)

def pick_col(df, preferred, candidates):
    # exact match first
    if preferred and preferred in df.columns:
        return preferred
    for c in candidates:
        if c in df.columns:
            return c
    # relaxed: remove non-alnum for matching
    def norm(s): return re.sub(r"[^a-z0-9]", "", str(s).lower())
    normmap = {col: norm(col) for col in df.columns}
    targets = ([preferred] if preferred else []) + list(candidates)
    for t in targets:
        key = norm(t)
        for col, nm in normmap.items():
            if nm == key:
                return col
    return None

INTAKE = pick_col(raw, None, ["Open Conf Date","Open Conf. Date","Open Conference Date"])
ZIPCOL = pick_col(raw, None, ["GeoZip","Geo Zip","Site Zip","Site  Zip"])
COUNTY = pick_col(raw, None, ["GeoCounty","Geo County","Site County","County"])

print("Resolved columns →", "intake:", INTAKE, "| zip:", ZIPCOL, "| county:", COUNTY)

if INTAKE is None:
    raise ValueError("Could not locate an intake date column on 'Complaints' with header=14.")

dt = pd.to_datetime(raw[INTAKE], errors="coerce")
mask_years = dt.dt.year.isin(YEARS)
cal_y = raw.loc[mask_years].copy()
cal_y["__intake"] = dt.loc[mask_years]

# ZIP→5 and geo merge
cal_y["ZIP"] = cal_y[ZIPCOL].astype(str).apply(zip5) if ZIPCOL else ""
merged = cal_y[["ZIP"]].rename(columns={"ZIP":"ZIP5"}).merge(geo, how="left", on="ZIP5")
merged.index = cal_y.index

# County resolution (B and 1)
county_from_agency = (cal_y[COUNTY] if COUNTY else "").fillna("").astype(str).str.strip()
county_from_zip = merged["GeoCounty"].fillna("").astype(str).str.strip()
cal_y["County_resolved"] = np.where(
    (cal_y["ZIP"] != "") & (county_from_zip != ""),
    county_from_zip,
    county_from_agency
)

# Rural flag + quick counts
cal_y["Is_Rural_Worker"] = cal_y["County_resolved"].apply(lambda c: "Y" if c in RURAL_COUNTIES else "N")

print("Cal/OSHA in YEARS:", len(cal_y))
print("\nTop counties:\n", cal_y["County_resolved"].value_counts().head(15))
print("\nRural flag counts:\n", cal_y["Is_Rural_Worker"].value_counts(dropna=False))


Resolved columns → intake: Open Conf Date | zip: GeoZip | county: GeoCounty
Cal/OSHA in YEARS: 144

Top counties:
 County_resolved
Los Angeles       32
Fresno            21
San Diego         12
Riverside          8
Tulare             8
Kern               8
Sacramento         7
Kings              5
Imperial           5
Contra Costa       5
Madera             4
Yolo               4
San Bernardino     4
Ventura            2
Stanislaus         2
Name: count, dtype: int64

Rural flag counts:
 Is_Rural_Worker
N    85
Y    59
Name: count, dtype: int64
